### Web scrapping the https://www.opensecrets.org/races/election starting from TN district 7

In [1]:
#Import the packages and library
import requests
from bs4 import BeautifulSoup as BS
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

%matplotlib inline

import seaborn as sns
import datetime

## Reading TN District 07

In [2]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'  #URL of the webpage

response = requests.get(URL)  #creating a response object pulling the URL
type(response)       # it is a requests.status.models.Response
response.status_code  #200 not broke read properly
response.text         # helps open the object to see it looks
soup = BS(response.text) #converts the response object in Beautiful object
#print(soup.prettify()) #to visualize how the conversion looks like and be able to read the class, tab etc where are the containers
soup.find('title')

<title>Tennessee District 07 2020 Race • OpenSecrets</title>

### we are have the soup so we can start finding the info and found out that Class 'Member--list-item' has all the members from the district contained in it.

In [3]:
#Finding all the members in the list item and 
list_item1 = soup.find('div', attrs={'class' : 'Members--list-item'}).text
list_item1

'\n\n\nMark Green (R) • Incumbent\n • Winner \n(69.9% of vote)\n\n\n\n\nRaised:\n$1,194,960\n\n\n\n\n\n\n\nSpent:\n$935,487\n\n\n\n\n\n\n\nCash on Hand:\n$287,889\n\n\n\n\n\n\n\n\nSource of Funds\n\n\n\nType of Contribution\nAmount\nPercentage\n\n\n\n\nSmall Individual Contributions (≤ $200)\n$0\n0.00%\n\n\nLarge Individual Contributions\n$819,151\n68.55%\n\n\nPAC Contributions*\n$171,900\n14.39%\n\n\nCandidate self-financing\n$0\n0.00%\n\n\nOther\n$203,909\n17.06%\n\n\n\nQuality of Disclosure\n\n\n\nType of Disclosure\nAmount\nPercentage\n\n\n\n\nFull Disclosure\n$1,061,024\n98.64%\n\n\nIncomplete Disclosure\n$2,676\n0.25%\n\n\nNo Disclosure\n$11,928\n1.11%\n\n\n\n\n'

#### List_item1 has many \n, , and dot that needes to be replaced so that we can run reggular expression on it so using the .replace function and assingn back to list_item1

In [4]:
list_item1 = list_item1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
list_item1

'   Mark Green (R)  Incumbent   Winner  (69.9% of vote)     Raised: $1194960        Spent: $935487        Cash on Hand: $287889         Source of Funds    Type of Contribution Amount Percentage     Small Individual Contributions (≤ $200) $0 0.00%   Large Individual Contributions $819151 68.55%   PAC Contributions* $171900 14.39%   Candidate self-financing $0 0.00%   Other $203909 17.06%    Quality of Disclosure    Type of Disclosure Amount Percentage     Full Disclosure $1061024 98.64%   Incomplete Disclosure $2676 0.25%   No Disclosure $11928 1.11%     '

#### Using regular expression and pulling out the candidate name, Party, %Votes they had, Dollars raised and Dollars Spent and figuring out if they are Winner or incumbent and putting 'Yes' if the expression is a MAtch  or 'No' if there is no match. We are creating empty list and appending the list when it moves from member to member.

In [5]:
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []


for list_item1 in soup.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_item1 = list_item1.text

    list_item1 = list_item1.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    
    
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_item1)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_item1)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_item1)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_item1)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_item1)[1])
    
    x = re.search(r"\bWinner", list_item1)  
    #the search makes the word match so it stops if no match so that is why weused the append YEs or No
    if x:
        Winner.append("Yes")
    else:
        Winner.append("No")
        
    y = re.search(r"\bIncumbent", list_item1)

    if y:
      Incumbent.append("Yes")
    else:
       Incumbent.append("No") 
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

['   Mark Green ', '             Kiran Sreepada ', '             Ronald Brown ', '             Scott Vieira Jr '] ['R', 'D', 'I', 'I'] ['69.9%', '27.3%', '2.2%', '0.6%'] ['1194960', '206644', '1750', '655'] ['935487', '207191', '0', '1049'] ['Yes', 'No', 'No', 'No'] ['Yes', 'No', 'No', 'No']


#### Now converting these list into a Dictionary object and using that to creat the Data Frame

In [6]:
df_TN7 = {'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
pd.DataFrame(df_TN7)

,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,Mark Green,R,69.9%,1194960,935487,Yes,Yes
1,Kiran Sreepada,D,27.3%,206644,207191,No,No
2,Ronald Brown,I,2.2%,1750,0,No,No
3,Scott Vieira Jr,I,0.6%,655,1049,No,No


### Trying to tanslate the above code for other districts in TN

#### There is pattern in website were 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N' where the id=TN is the stae and 07 is the district so we are going to follow the logic and creat a variabe called state feed the state code to it and a d_num variable which will change adding 1 from the previos district number and starts with 01

In [7]:
state = 'TN'
d_num = 1

#this URL is different for races/summary we need the races/candidates instead for picking the candidates
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id='+str(state)+'0'+str(d_num)+'&spec=N'  #URL of the webpage
#URLX = 'https://www.opensecrets.org/races/summary?cycle=2022&id=TN02&spec=N'

#URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N'
URLOX1 = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN06&spec=N'

URLOX = 'https://www.opensecrets.org/races/candidates?cycle=2020&id='+str(state)+'0'+str(d_num)+'&spec=N'
print(URLOX)

responseX = requests.get(URLOX)  #creating a response object pulling the URL
type(responseX)       # it is a requests.status.models.Response

responseX.status_code  #200 not broke read properly
responseX.text         # helps open the object to see it looks
soupX = BS(responseX.text, 'html.parser') #converts the response object in Beautiful object
#print(soup.prettify())
soupX.find('title')

list_itemX = soupX.find('div', attrs={'class' : 'Members--list-item'}).text
#list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")

list_itemX



https://www.opensecrets.org/races/candidates?cycle=2020&id=TN01&spec=N


'\n\n\n\t\t\t\t\t\t\t\t\t\tDiana Harshbarger (R)\n • Winner \n(74.8% of vote)\n\n\n\n\nRaised:\n$2,126,946\n\n\n\n\n\n\n\nSpent:\n$1,869,100\n\n\n\n\n\n\n\nCash on Hand:\n$257,846\n\n\n\n\n\n\n\n\nSource of Funds\n\n\n\nType of Contribution\nAmount\nPercentage\n\n\n\n\nSmall Individual Contributions (≤ $200)\n$44,239\n2.08%\n\n\nLarge Individual Contributions\n$315,489\n14.83%\n\n\nPAC Contributions*\n$222,800\n10.48%\n\n\nCandidate self-financing\n$1,461,293\n68.70%\n\n\nOther\n$83,124\n3.91%\n\n\n\nQuality of Disclosure\n\n\n\nType of Disclosure\nAmount\nPercentage\n\n\n\n\nFull Disclosure\n$185,473\n94.33%\n\n\nIncomplete Disclosure\n$2,800\n1.42%\n\n\nNo Disclosure\n$8,350\n4.25%\n\n\n\n\n'

In [10]:
Name = []
Party = []
pct_votes = []
Raised = []
Spent = []
Winner = []
Incumbent = []


for list_itemX in soupX.findAll('div', attrs={'class' : 'Members--list-item'}):
    
    list_itemX = list_itemX.text

    list_itemX = list_itemX.replace("\n", " ").replace(",", "").replace("\t", " ").replace("•", "")
    
    
    Name.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[1])
   
    Party.append(re.search(r"(.+)\(([A-Z])\)", list_itemX)[2])
    
    pct_votes.append(re.search("\d+\.\d+\%", list_itemX)[0])
    
    Raised.append(re.search(r"\bRaised\b\:\W\$(\d*)", list_itemX)[1])
    
    Spent.append(re.search(r"\bSpent\b\:\W\$(\d*)", list_itemX)[1])
    
    x = re.search(r"\bWinner", list_itemX)  
    #the search makes the word match so it stops if no match so that is why weused the append YEs or No
    if x:
        Winner.append("Yes")
    else:
        Winner.append("No")
        
    y = re.search(r"\bIncumbent", list_itemX)

    if y:
      Incumbent.append("Yes")
    else:
       Incumbent.append("No") 
print(Name, Party, pct_votes, Raised, Spent, Winner, Incumbent)

df_TN1 = {'CandidateName': Name,'Party': Party,  'PercentVotes' : pct_votes, 'FundsRaised': Raised, 'MoneySpent':Spent, 
          'Winner': Winner, 'Incumbent': Incumbent}
pd.DataFrame(df_TN1)

['             Diana Harshbarger ', '             Blair Nicole Walsingham '] ['R', 'D'] ['74.8%', '22.4%'] ['2126946', '140209'] ['1869100', '134995'] ['Yes', 'No'] ['No', 'No']


,CandidateName,Party,PercentVotes,FundsRaised,MoneySpent,Winner,Incumbent
0,Diana Harshbarger,R,74.8%,2126946,1869100,Yes,No
1,Blair Nicole Walsingham,D,22.4%,140209,134995,No,No
